# Final Report Without Augmentation

In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1
set_session(tf.Session(config=config))

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw


# Root directory of the project
ROOT_DIR = os.path.abspath("../..")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = "mask_rcnn_coco.h5"

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [3]:
ROOT_DIR

'c:\\abhilash'

In [4]:
#from idvscript import main
import ivddataset

# Experiment Configuration

In [5]:
class IDVConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "idv"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    
    # Number of classes (including background)
    NUM_CLASSES = 1 + 4  # Background + balloon

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 50
    
    #RPN_ANCHOR_SCALES = (4,8,32, 64, 128)
    
    # Loss weights for more precise optimization.
    # Can be used for R-CNN training setup.
    '''LOSS_WEIGHTS = {
        "rpn_class_loss": 1.,
        "rpn_bbox_loss": 1.,
        "mrcnn_class_loss": 1.,
        "mrcnn_bbox_loss": 1.,
        "mrcnn_mask_loss": 1.
    }'''

    # Skip detections with < 60% confidence
    DETECTION_MIN_CONFIDENCE = 0.6

config = IDVConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.6
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                17
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

In [6]:
import h5py

# Instantiate Model

In [7]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)

# Load pretrained weights
model_path = COCO_WEIGHTS_PATH
model.load_weights(model_path, by_name=True, exclude=[
           "mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])

# Load Training and Validation Dataset

In [8]:
# Load training dataset
dataset_train = ivddataset.IDVDataset()
dataset_train.load_idv(dataset_dir=os.path.join("train"))
dataset_train.prepare()

# Validation dataset
dataset_val = ivddataset.IDVDataset()
dataset_val.load_idv(dataset_dir=os.path.join("val"))
dataset_val.prepare()

# Train

In [9]:
# *** This training schedule is an example. Update to your needs ***

# Training - Stage 1
print("Training network heads")
model.train(dataset_train,dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=10,
            layers='heads')

# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train,dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=20,
            layers='4+')

# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train,dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=30,
            layers='all')

Training network heads
Instructions for updating:
Use the retry module or similar alternatives.

Starting at epoch 0. LR=0.001

Checkpoint Path: c:\abhilash\logs\idv20180629T0406\mask_rcnn_idv_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_

C:\Anaconda\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
50/50 [==============================] - 360s 7s/step - loss: 1.8545 - rpn_class_loss: 6.4993e-04 - rpn_bbox_loss: 0.2113 - mrcnn_class_loss: 0.1779 - mrcnn_bbox_loss: 0.5961 - mrcnn_mask_loss: 0.8685 - val_loss: 1.0571 - val_rpn_class_loss: 0.0021 - val_rpn_bbox_loss: 0.3218 - val_mrcnn_class_loss: 0.0889 - val_mrcnn_bbox_loss: 0.2940 - val_mrcnn_mask_loss: 0.3503
Epoch 2/10
50/50 [==============================] - 321s 6s/step - loss: 0.6969 - rpn_class_loss: 6.9863e-04 - rpn_bbox_loss: 0.1464 - mrcnn_class_loss: 0.0736 - mrcnn_bbox_loss: 0.1958 - mrcnn_mask_loss: 0.2804 - val_loss: 0.7597 - val_rpn_class_loss: 0.0018 - val_rpn_bbox_loss: 0.2682 - val_mrcnn_class_loss: 0.0831 - val_mrcnn_bbox_loss: 0.1961 - val_mrcnn_mask_loss: 0.2105
Epoch 3/10
50/50 [==============================] - 315s 6s/step - loss: 0.5430 - rpn_class_loss: 6.5794e-04 - rpn_bbox_loss: 0.1630 - mrcnn_class_loss: 0.0630 - mrcnn_bbox_loss: 0.1330 - mrcnn_mask_loss: 0.1834 - val_loss: 0.6282 - val_rpn_c

C:\Anaconda\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 11/20
50/50 [==============================] - 327s 7s/step - loss: 0.2796 - rpn_class_loss: 9.2639e-04 - rpn_bbox_loss: 0.1037 - mrcnn_class_loss: 0.0402 - mrcnn_bbox_loss: 0.0428 - mrcnn_mask_loss: 0.0919 - val_loss: 0.4899 - val_rpn_class_loss: 0.0013 - val_rpn_bbox_loss: 0.2646 - val_mrcnn_class_loss: 0.0686 - val_mrcnn_bbox_loss: 0.0556 - val_mrcnn_mask_loss: 0.0999
Epoch 12/20
50/50 [==============================] - 307s 6s/step - loss: 0.3020 - rpn_class_loss: 7.9118e-04 - rpn_bbox_loss: 0.1371 - mrcnn_class_loss: 0.0380 - mrcnn_bbox_loss: 0.0441 - mrcnn_mask_loss: 0.0820 - val_loss: 0.4928 - val_rpn_class_loss: 0.0011 - val_rpn_bbox_loss: 0.2842 - val_mrcnn_class_loss: 0.0468 - val_mrcnn_bbox_loss: 0.0703 - val_mrcnn_mask_loss: 0.0904
Epoch 13/20
50/50 [==============================] - 308s 6s/step - loss: 0.2302 - rpn_class_loss: 4.4079e-04 - rpn_bbox_loss: 0.0712 - mrcnn_class_loss: 0.0507 - mrcnn_bbox_loss: 0.0372 - mrcnn_mask_loss: 0.0707 - val_loss: 0.4113 - val_rp

C:\Anaconda\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 21/30
36/50 [====================>.........] - ETA: 1:09 - loss: 0.0950 - rpn_class_loss: 2.9860e-04 - rpn_bbox_loss: 0.0158 - mrcnn_class_loss: 0.0155 - mrcnn_bbox_loss: 0.0133 - mrcnn_mask_loss: 0.0501

ResourceExhaustedError: OOM when allocating tensor with shape[2,256,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_2/SGD/gradients/roi_align_classifier/CropAndResize_grad/CropAndResizeGradImage = CropAndResizeGradImage[T=DT_FLOAT, _class=["loc:@roi_align_classifier/CropAndResize"], method="bilinear", _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_2/SGD/gradients/roi_align_classifier/CropAndResize_grad/CropAndResizeGradImage/grads, roi_align_classifier/GatherNd, roi_align_classifier/StopGradient_1/_12315, training_2/SGD/gradients/roi_align_classifier/CropAndResize_grad/Shape-0-3-VecPermuteNCHWToNHWC-LayoutOptimizer/_12425)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: training_2/SGD/gradients/mrcnn_mask_conv1/convolution_grad/Conv2DBackpropInput-0-0-TransposeNCHWToNHWC-LayoutOptimizer/